In [24]:
#modelo de machine learning supervisionado
#utilizando dataset ja classificado

In [108]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score

In [35]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/wagui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/wagui/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /home/wagui/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/wagui/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/wagui/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [67]:
pd.options.display.max_colwidth = 100

In [2]:
df = pd.read_csv('../raw_data/Tweets_Mg.csv', encoding='utf-8')

In [3]:
df.head()

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#balanceamento do dataset

In [20]:
df.Classificacao.value_counts()

Positivo    3300
Neutro      2453
Negativo    2446
Name: Classificacao, dtype: int64

In [27]:
#removendo comentarios duplicados

In [28]:
df.drop_duplicates(['Text'], inplace=True)

In [ ]:
#Separando features e target

In [29]:
X = df['Text']
y = df['Classificacao']

In [30]:
#data cleaning...

In [98]:
from sklearn.base import BaseEstimator, TransformerMixin
import string
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
class CleaningEncoder(BaseEstimator, TransformerMixin):
    '''
    Receives raw text data from the tweets and returns clean, ready to process data:
    turns all into lower case;
    removes punctuation;
    removes stopwords;
    removes numbers;
    removes users' handles
    
    '''
    def __init__(self):
        pass
                        
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        #assert isinstance(X, pd.DataFrame)
        X_ = X.copy()      
        
        #removin NaN values
        X_ = X_.fillna('nenhuma descrição')
        
        #removing links
        X_ = X_.apply(lambda x: re.sub(r"http\S+", "", x))
        X_ = X_.apply(lambda x: re.sub(r"www.\S+", "", x))
                
        #removing punctuation from each tweet
        new_punc = list(string.punctuation)
        del new_punc[2]
        del new_punc[-11]
        for punctuation in new_punc:
            X_ = X_.str.replace(punctuation, '')
            
        #removing numbers
        X_ = X_.str.replace('\d+', '')
        
        #tokenizing - removes handles, applies lowercase, keeps #, shortens letter repetitions to three
        #ex: kkkkk, kkkkkk, kkkkkkkkk = kkk
        tkn = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
        X_ = X_.apply(lambda x: tkn.tokenize(x))
        
        #removing stopwords
        stop_words = stopwords.words('portuguese')
        stop_words.remove('não')
        addicional = [
            'd', 'ta', 'q', 'tah', 'tao', 'eh', 'vc', 'voce',
            'pq', 'quedê', 'mto', 'mt', 'bj', 'bjs',
            'b', 'sao', 'axo', 'mano', 'ae', 'neh', 'aí',
            'kkk', 'porque', 'né', 'no', 'iai', 'tbm', 'msm', 'jah']
        stop_words.extend(addicional)
        
        X_ = X_.apply(lambda x: ' '.join(word for word in x if word not in (stop_words)))
        
        return X_

In [105]:
cleaner = CleaningEncoder()
X_vectorized = cleaner.transform(X)
X_vectorized[23]

/tmp/ipykernel_7719/546299116.py:38: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_ = X_.str.replace(punctuation, '')
/tmp/ipykernel_7719/546299116.py:41: FutureWarning: The default value of regex will change from True to False in a future version.
  X_ = X_.str.replace('\d+', '')


'praia estação lota neste sábado verão bh gerais estado minas'

In [92]:
#tokenization

In [106]:
#bag of words
vectorizer = CountVectorizer()

X_bow = vectorizer.fit_transform(X_vectorized)

X_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
#modelo 

In [124]:
nb_model = MultinomialNB()
nb_model.fit(X_bow, y)

MultinomialNB()

In [125]:
accuracy = cross_val_score(nb_model, X_bow, y, cv=10).mean()
accuracy

0.8640543760831889